<a href="https://colab.research.google.com/github/MWaser/BurstIQ/blob/main/OPAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Econometrica vs. Public OP Data Analysis

<b>Results TL;DR</b><ol>
<li>EM has FEWER Products per year than the published OPA data</li>
<li>EM has FEWER Payments per year than the published OPA data</li>
<li><b><i>EM has records of OPA Payments that DO NOT EXIST in the published OPA data</b></i></li></b><ol>

<b>This notebook must be downloaded to a whitelisted machine to re-run queries.  You must also fill in your SQL userID and password</b>

In [ ]:
userID = ""
password = ""

## Notebook Initialization

In [ ]:
from tabulate import tabulate
import pyodbc

def executeQuery(query, hasResults = 'yes', close = 'yes'):
   conn = pyodbc.connect("DRIVER={ODBC Driver 17 for SQL Server};server=92.204.135.191;UID=" + userID + ";PWD=" + password + ";database=CTrial;");
   conn
   cursor = conn.cursor()
   for q in query:
      cursor.execute(q)
   # if (hasResults == 'yes'):
   result = cursor.fetchall()
   field_names = [i[0] for i in cursor.description]
   print(tabulate(result, headers=field_names, tablefmt='psql'))
   #if (close == 'yes'):        # remember - closing will delete temporary tables
   cursor.close()
   conn.close()


## Initial Product Row Count Analysis

**EM Data has one product per row in the table EM_OPProducts**

In [ ]:
executeQuery(["SELECT PYMT_PGM_YR_NUM, PAYMENT_TYPE, COUNT(1) AS NumRows from EM_OPProducts GROUP BY PYMT_PGM_YR_NUM, PAYMENT_TYPE"]);

+-------------------+----------------+-----------+
|   PYMT_PGM_YR_NUM | PAYMENT_TYPE   |   NumRows |
|-------------------+----------------+-----------|
|              2016 | General        |  14920281 |
|              2017 | General        |  14312008 |
|              2018 | General        |  13296766 |
|              2019 | General        |  12520056 |
|              2020 | General        |   6922208 |
|              2021 | General        |  13563099 |
|              2022 | General        |  15286759 |
|              2016 | Research       |    500645 |
|              2017 | Research       |    497771 |
|              2018 | Research       |    455670 |
|              2019 | Research       |    430928 |
|              2020 | Research       |    413090 |
|              2021 | Research       |    486507 |
|              2022 | Research       |    585837 |
+-------------------+----------------+-----------+


<b>OPA public data has one payment with multiple products per row split into two tables based upon PAYMENT_TYPE<br>
We have denormalized to one product per row into the tables OP_GeneralProducts and OP_ResearchProducts</b>

In [ ]:
executeQuery(["SELECT Program_Year, COUNT(1) AS NumRows FROM OP_GENERAL g INNER JOIN OP_GeneralProducts p ON p.Record_ID = g.Record_ID \
GROUP BY Program_Year ORDER BY Program_Year"])

+----------------+-----------+
|   Program_Year |   NumRows |
|----------------+-----------|
|           2016 |  15318303 |
|           2017 |  14675825 |
|           2018 |  13644942 |
|           2019 |  12827297 |
|           2020 |   7076265 |
|           2021 |  13791260 |
|           2022 |  15492732 |
+----------------+-----------+


In [ ]:
executeQuery(["SELECT Program_Year, COUNT(1) AS NumRows FROM OP_Research r INNER JOIN OP_ResearchProducts p ON p.Record_ID = r.Record_ID \
GROUP BY Program_Year ORDER BY Program_Year"])

+----------------+-----------+
|   Program_Year |   NumRows |
|----------------+-----------|
|           2016 |    772824 |
|           2017 |    759411 |
|           2018 |    696036 |
|           2019 |    598098 |
|           2020 |    589042 |
|           2021 |    664399 |
|           2022 |    901131 |
+----------------+-----------+


<div class="alert alert-block alert-info">
    <b>EM has FEWER products per year than the published OPA data</b>
</div>

## Initial Payment Row Count & Subset Analysis

In [ ]:
# This block takes about 6 minutes to run
executeQuery(["SELECT DISTINCT PYMT_PGM_YR_NUM, OPEN_PYMTS_SYS_ID INTO #EM_OPA_SysIDs from EM_OPProducts", \
              "SELECT PYMT_PGM_YR_NUM, COUNT(1) AS NumRows FROM #EM_OPA_SysIDs GROUP BY PYMT_PGM_YR_NUM"])

+-------------------+-----------+
|   PYMT_PGM_YR_NUM |   NumRows |
|-------------------+-----------|
|              2016 |  11271671 |
|              2017 |  11060382 |
|              2018 |  10534443 |
|              2019 |  10171195 |
|              2020 |   5810607 |
|              2021 |  11231472 |
|              2022 |  12798263 |
+-------------------+-----------+


In [ ]:
executeQuery(["SELECT Program_Year, COUNT(1) AS NumRows FROM OP_General GROUP BY Program_Year ORDER BY Program_Year"])
executeQuery(["SELECT Program_Year, COUNT(1) AS NumRows FROM OP_Research GROUP BY Program_Year ORDER BY Program_Year"])

+----------------+-----------+
|   Program_Year |   NumRows |
|----------------+-----------|
|           2016 |  11713844 |
|           2017 |  11423318 |
|           2018 |  10936295 |
|           2019 |  10551081 |
|           2020 |   5836625 |
|           2021 |  11512430 |
|           2022 |  13148520 |
+----------------+-----------+
+----------------+-----------+
|   Program_Year |   NumRows |
|----------------+-----------|
|           2016 |    880282 |
|           2017 |    819771 |
|           2018 |    793981 |
|           2019 |    712806 |
|           2020 |    669348 |
|           2021 |    708938 |
|           2022 |    953320 |
+----------------+-----------+


<div class="alert alert-block alert-info">
    <b>EM has a FEWER OPA Payments per year than the published OPA data</b>
</div>

In [ ]:
executeQuery(["SELECT PYMT_PGM_YR_NUM, COUNT(1) UnmatchedRows FROM EM_OPProducts e WHERE PAYMENT_TYPE = 'General' AND \
              NOT EXISTS(SELECT 1 FROM OP_General WHERE Record_ID = e.OPEN_PYMTS_SYS_ID) GROUP BY PYMT_PGM_YR_NUM"])

+-------------------+-----------------+
|   PYMT_PGM_YR_NUM |   UnmatchedRows |
|-------------------+-----------------|
|              2016 |               1 |
|              2017 |            4453 |
|              2018 |            1015 |
|              2019 |            1215 |
|              2020 |            1349 |
|              2021 |           42476 |
|              2022 |          100392 |
+-------------------+-----------------+


In [ ]:
executeQuery(["SELECT PYMT_PGM_YR_NUM, COUNT(1) UnmatchedRows FROM EM_OPProducts e WHERE PAYMENT_TYPE = 'Research' AND \
              NOT EXISTS(SELECT 1 FROM OP_Research WHERE Record_ID = e.OPEN_PYMTS_SYS_ID) GROUP BY PYMT_PGM_YR_NUM"])

+-------------------+-----------------+
|   PYMT_PGM_YR_NUM |   UnmatchedRows |
|-------------------+-----------------|
|              2016 |               1 |
|              2017 |             122 |
|              2019 |            5477 |
|              2020 |           14754 |
|              2021 |           12478 |
|              2022 |           16729 |
+-------------------+-----------------+


<div class="alert alert-block alert-info">
    <b>EM has records of OPA Payments that DO NOT EXIST in the published OPA data</b>
</div>